In [1]:
import numpy as np
import pandas as pd
import os

PATH = "/kaggle/input/tsp-cv"
PATH_TRAIN = os.path.join(PATH, "train.csv")

In [2]:
# What version of Python do you have?
import sys

import tensorflow.keras
import pandas as pd
import sklearn as sk
import tensorflow as tf

print(f"Tensor Flow Version: {tf.__version__}")
print(f"Keras Version: {tensorflow.keras.__version__}")
print()
print(f"Python {sys.version}")
print(f"Pandas {pd.__version__}")
print(f"Scikit-Learn {sk.__version__}")
print("GPU is", "available" if tf.test.is_gpu_available() \
      else "NOT AVAILABLE")

Tensor Flow Version: 2.6.2
Keras Version: 2.6.0

Python 3.7.12 | packaged by conda-forge | (default, Oct 26 2021, 06:08:53) 
[GCC 9.4.0]
Pandas 1.3.5
Scikit-Learn 1.0.1
GPU is available


2022-04-17 20:04:46.977977: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-04-17 20:04:47.046363: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-17 20:04:47.142813: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-17 20:04:47.143605: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA 

In [3]:
df_train = pd.read_csv(PATH_TRAIN)

In [4]:
PATH_test = os.path.join(PATH, "test.csv")
df_test = pd.read_csv(PATH_test)

In [5]:
TRAIN_PCT = 0.9
TRAIN_CUT = int(len(df_train) * TRAIN_PCT)

df_train_cut = df_train[0:TRAIN_CUT]
df_validate_cut = df_train[TRAIN_CUT:]

print(f"Training size: {len(df_train_cut)}")
print(f"Validate size: {len(df_validate_cut)}")

Training size: 14416
Validate size: 1602


In [6]:
import tensorflow as tf
import keras_preprocessing
from keras_preprocessing import image
from keras_preprocessing.image import ImageDataGenerator

WIDTH = 256
HEIGHT = 256

training_datagen = ImageDataGenerator(
  rescale = 1./255,
  horizontal_flip=True,
  #vertical_flip=True,
  fill_mode='nearest')

train_generator = training_datagen.flow_from_dataframe(
        dataframe=df_train_cut,
        directory=PATH,
        x_col="filename",
        y_col="distance",
        target_size=(HEIGHT, WIDTH),
        batch_size=32, 
        class_mode='raw')

validation_datagen = ImageDataGenerator(rescale = 1./255)

val_generator = validation_datagen.flow_from_dataframe(
        dataframe=df_validate_cut,
        directory=PATH,
        x_col="filename",
        y_col="distance",
        target_size=(HEIGHT, WIDTH),
        batch_size=256, 
        class_mode='raw')

Found 14416 validated image filenames.
Found 1602 validated image filenames.


## Transfer Learning

We will now use a ResNet neural network as a basis for our neural network.  We will redefine both the input shape and output of the ResNet model, so we will not transfer the weights.  

In [7]:
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.layers import Input

input_tensor = Input(shape=(HEIGHT, WIDTH, 3))

base_model = ResNet50(
    include_top=False, weights=None, input_tensor=input_tensor,
    input_shape=None)

#base_model.summary()

2022-04-17 20:04:55.990690: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-17 20:04:55.991575: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-17 20:04:55.992217: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-17 20:04:55.993050: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-17 20:04:55.993712: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from S

In [8]:
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

x=base_model.output
x=GlobalAveragePooling2D()(x)
x=Dense(1024,activation='relu')(x) 
x=Dense(1024,activation='relu')(x) 
model=Model(inputs=base_model.input,outputs=Dense(1)(x))

In [9]:
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.metrics import RootMeanSquaredError

# Important, calculate a valid step size for the validation dataset
STEP_SIZE_VALID=val_generator.n//val_generator.batch_size

model.compile(loss = 'mean_squared_error', optimizer='adam', metrics=[RootMeanSquaredError(name="rmse")])
monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=10, verbose=1, mode='auto',
        restore_best_weights=True)

history = model.fit(train_generator, epochs=100, steps_per_epoch=50, 
                    validation_data = val_generator, callbacks=[monitor],
                    verbose = 1, validation_steps=STEP_SIZE_VALID)

2022-04-17 20:04:57.784896: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/100


2022-04-17 20:05:03.573881: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


50/50 [==============================] - 57s 916ms/step - loss: 411571040.0000 - rmse: 20287.2129 - val_loss: 69000552448.0000 - val_rmse: 262679.5625
Epoch 2/100
50/50 [==============================] - 36s 715ms/step - loss: 118501824.0000 - rmse: 10885.8545 - val_loss: 1870177280.0000 - val_rmse: 43245.5469
Epoch 3/100
50/50 [==============================] - 35s 696ms/step - loss: 91910776.0000 - rmse: 9587.0107 - val_loss: 650313024.0000 - val_rmse: 25501.2363
Epoch 4/100
50/50 [==============================] - 34s 687ms/step - loss: 107847976.0000 - rmse: 10384.9883 - val_loss: 742528192.0000 - val_rmse: 27249.3711
Epoch 5/100
50/50 [==============================] - 33s 663ms/step - loss: 74490712.0000 - rmse: 8630.7998 - val_loss: 764117952.0000 - val_rmse: 27642.6836
Epoch 6/100
50/50 [==============================] - 34s 670ms/step - loss: 93158304.0000 - rmse: 9651.8545 - val_loss: 779038528.0000 - val_rmse: 27911.2617
Epoch 7/100
50/50 [==============================] - 3

In [10]:
submit_datagen = ImageDataGenerator(rescale = 1./255)

submit_generator = submit_datagen.flow_from_dataframe(
        dataframe=df_test,
        directory=PATH,
        x_col="filename",
        batch_size = 1,
        shuffle = False,
        target_size=(HEIGHT, WIDTH),
        class_mode=None)

submit_generator.reset()
pred = model.predict(submit_generator,steps=len(df_test))

Found 4005 validated image filenames.


In [11]:
pred

array([[ 8544.627],
       [59375.793],
       [11630.045],
       ...,
       [13292.574],
       [15693.588],
       [11921.466]], dtype=float32)

In [12]:
df_submit = pd.DataFrame({"id":df_test['id'],'distance':pred[:,0].flatten()})
df_submit.to_csv("./resnet50.csv",index = False)